# Music recommendation system

## ZHONG JING 20654235

To develop a music recommendation model, after the feature engineering process, I mainly try three models Light GBM, TPOT autoML and adaboost. It turns out that Light GBM is the best one.

In [1]:
import pandas as pd
import numpy as np
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from tpot import TPOTClassifier

C:\Users\13269\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\13269\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


## data loading

Original data include five datasets, train, test, members, songs and song_extra_info.

In [2]:
dataPath = "C:/Users/13269/Desktop/20 Spring Semester/MAFS6010S (L1) Machine Learning and its Applications/project/kkbox/data/"

## load data
train = pd.read_csv(dataPath + 'train.csv', dtype={'msno' : 'category',
                                                   'song_id' : 'category',
                                                   'source_system_tab' : 'category',
                                                   'source_screen_name' : 'category',
                                                   'source_type' : 'category'})
test = pd.read_csv(dataPath + 'test.csv', dtype={'msno' : 'category',
                                                 'song_id' : 'category',
                                                 'source_system_tab' : 'category',
                                                 'source_screen_name' : 'category',
                                                 'source_type' : 'category'})
members = pd.read_csv(dataPath + 'members.csv', dtype={'msno' : 'category',
                                                       'city' : 'category',
                                                       #'bd' : np.uint8,
                                                       'gender' : 'category',
                                                       'registered_via' : 'category'},
                      parse_dates=['registration_init_time', 'expiration_date'])
songs = pd.read_csv(dataPath + 'songs.csv', dtype={'song_id' : 'category',
                                                   'genre_ids': 'category',
                                                   'artist_name' : 'category',
                                                   'composer' : 'category',
                                                   'lyricist' : 'category',
                                                   'language' : 'category'})
song_extra_info = pd.read_csv(dataPath + 'song_extra_info.csv', dtype={'song_id' : 'category'})

Since songs data are too large, to make the following data merge procedures faster, I only keep songs information that appear in the train or test data.

In [3]:
## filter for song's information
song_id = pd.concat([train.song_id, test.song_id]).unique()
songs = songs[songs.song_id.isin(song_id)]
song_extra_info = song_extra_info[song_extra_info.song_id.isin(song_id)]

## feature engineering

Feature engineering is an important procedure in modeling. In these datasets, I can derive some other deeper variables based on the original data. <br>
For members data, membership_days can be derived from expiration_date and registration_init_time, also both expiration_date and registration_init_time can be splited into year, month and day variables to replace with the date variable. <br>
For song_extra_info data, there is a isrc - International Standard Recording Code, which we can extract country code, registrant code and song_year. 

In [4]:
## deal with members data
members['membership_days'] = members['expiration_date'].subtract(members['registration_init_time']).dt.days.astype(int)
members['registration_year'] = members['registration_init_time'].dt.year
members['registration_month'] = members['registration_init_time'].dt.month
members['registration_day'] = members['registration_init_time'].dt.day
members['expiration_year'] = members['expiration_date'].dt.year
members['expiration_month'] = members['expiration_date'].dt.month
members['expiration_day'] = members['expiration_date'].dt.day
members = members.drop(['registration_init_time','expiration_date'], axis=1)

In [5]:
## deal with songs_extra data
def isrc_to_year(isrc):
    if type(isrc) == str:
        if int(isrc[5:7]) > 17:
            return 1900 + int(isrc[5:7])
        else:
            return 2000 + int(isrc[5:7])
    else:
        return np.nan

song_extra_info['country_code'] = song_extra_info['isrc'].apply(lambda x: str(x)[0:2])
song_extra_info['registrant_code'] = song_extra_info['isrc'].apply(lambda x: str(x)[2:5])
song_extra_info['song_year'] = song_extra_info['isrc'].apply(isrc_to_year)
song_extra_info.drop(['isrc', 'name'], axis = 1, inplace = True)

Merge all these data with key of msno and song_id.

In [6]:
## merge data
train = train.merge(members, on='msno', how='left')
train = train.merge(songs, on='song_id', how='left')
train = train.merge(song_extra_info, on = 'song_id', how = 'left')

test = test.merge(members, on='msno', how='left')
test = test.merge(songs, on='song_id', how='left')
test = test.merge(song_extra_info, on = 'song_id', how = 'left')

## delete original data
del members, songs, song_extra_info

All the string variables will be transform into categorical and interpolate missing value with level of 'no data'. For the numerical variables, missing data will be interpolate with 0.

In [7]:
## Converting object types to categorical
factor_Y = 'target'
factor_X = list(train.columns)
factor_X.remove(factor_Y)
#factor_process = list(set(factor_X).difference(['msno', 'song_id']))
for col in factor_X:
    if train[col].dtype == object:
        train[col] = train[col].astype('category')
        #test[col] = test[col].astype('category')

In [8]:
## deal with NA
for col in factor_X:
    # Replace NA for category
    if str(train[col].dtype) == 'category':
        train[col] = train[col].cat.add_categories(['no data'])
        train[col].fillna('no data', inplace=True)
        #test[col] = test[col].cat.add_categories(['no data'])
        #test[col].fillna('no data', inplace=True)
    # Replace NA for other
    else:
        train[col] = train[col].fillna(value=0)
        #test[col] = test[col].fillna(value=0)

Since the original test dataset do not have the actual target, so we have no idea of the accuracy of prediction on test data. In this case I will separate 20% of train data as test data to test the performance of models and use the rest of data to train the model.

In [9]:
## data split
X_train, X_test, y_train, y_test = train_test_split(train[factor_X], train[factor_Y], test_size=0.2, random_state=0)

After data split, I take all information in the training dataset as known, and take all information in the testing dataset as unknown. Based on the known historical information, I can create more features in terms of users and songs. Variable song_count means the number of songs that every user has heared and variable msno_count means the number of users that have heared of this song.

In [10]:
## more features
msno_song_count = X_train.groupby(['msno'])['song_id'].count().reset_index()
msno_song_count.columns = ['msno', 'song_count']
song_msno_count = X_train.groupby(['song_id'])['msno'].count().reset_index()
song_msno_count.columns = ['song_id', 'msno_count']

Then I want to try some more features such as the most preferred genre category, artist, composer, lyricist and songs language of every single user. However the computation of these features are too time consuming... So I have to skip them.

In [11]:
# msno_genre_count = X_train.groupby(['msno','genre_ids']).count().reset_index()
# msno_artist_count = X_train.groupby(['msno','artist_name']).count().reset_index()
# msno_composer_count = X_train.groupby(['msno','composer']).count().reset_index()
# msno_lyricist_count = X_train.groupby(['msno','lyricist']).count().reset_index()
# msno_language_count = X_train.groupby(['msno','language']).count().reset_index()

In [12]:
## merge new features
X_train = X_train.merge(msno_song_count, on='msno', how='left')
X_train = X_train.merge(song_msno_count, on='song_id', how='left')
X_test = X_test.merge(msno_song_count, on='msno', how='left')
X_test = X_test.merge(song_msno_count, on='song_id', how='left')

#test = test.merge(msno_song_count, on='msno', how='left')
#test = test.merge(song_msno_count, on='song_id', how='left')

In [13]:
## renew features
factor_X = list(X_train.columns)
#factor_X = list(set(factor_X).difference(['msno', 'song_id']))

Until now we have finished all the feature engineering part and get the final wide table for modeling. The following table shows the variables' type in our data, including 15 categorical and 12 numerical variables.

In [14]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5901934 entries, 0 to 5901933
Data columns (total 27 columns):
msno                  category
song_id               category
source_system_tab     category
source_screen_name    category
source_type           category
city                  category
bd                    int64
gender                category
registered_via        category
membership_days       int32
registration_year     int64
registration_month    int64
registration_day      int64
expiration_year       int64
expiration_month      int64
expiration_day        int64
song_length           float64
genre_ids             category
artist_name           category
composer              category
lyricist              category
language              category
country_code          category
registrant_code       category
song_year             float64
song_count            int64
msno_count            int64
dtypes: category(15), float64(2), int32(1), int64(9)
memory usage: 776.4 MB


## model 1 Light GBM

First, let's try Light GBM model. Light GBM is a fast, distributed, high-performance gradient boosting framework. Unlike other boosting algorithms it splits the trees leafwise and not level wise. It trains faster(on larger datasets) compared to other boosting algorithms like XGBoost. It uses leaf wise splitting instead of level wise splitting. Leaf wise splitting may lead to overfitting. This can be avoided by specifying tree-specific hyper parameters like max depth. <br>
To make training faster and avoid overfitting, I set learning to be 0.2, bagging_fraction to be 0.8, feature_fraction to be 0.8, num of leaves to be 100, iteration times to be 100. Here our metric is auc since it is a classification problem.

In [15]:
lgb_X_train, lgb_X_val, lgb_y_train, lgb_y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
lgb_train = lgb.Dataset(lgb_X_train, lgb_y_train)
lgb_val = lgb.Dataset(lgb_X_val, lgb_y_val)
lgb_test = lgb.Dataset(X_test, y_test)

In [16]:
params = {
        'objective': 'binary',
        'boosting': 'gbdt',
        'learning_rate': 0.2,
        'verbose': 0,
        'num_leaves': 100,
        'bagging_fraction': 0.8,
        'bagging_freq': 1,
        'bagging_seed': 1,
        'feature_fraction': 0.8,
        'feature_fraction_seed': 1,
        'max_bin': 256,
        'num_rounds': 200,
        'metric' : 'auc'
    }

In [17]:
lgbm_model = lgb.train(params, train_set = lgb_train, valid_sets = lgb_val, verbose_eval=5, early_stopping_rounds=5)
#lgbm_model = lgb.train(params, train_set = lgb_train, verbose_eval=5)

D:\Anaconda3\lib\site-packages\lightgbm\engine.py:99: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
D:\Anaconda3\lib\site-packages\lightgbm\basic.py:668: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


Training until validation scores don't improve for 5 rounds.
[5]	valid_0's auc: 0.744038
[10]	valid_0's auc: 0.754057
[15]	valid_0's auc: 0.76693
[20]	valid_0's auc: 0.776095
[25]	valid_0's auc: 0.781963
[30]	valid_0's auc: 0.785361
[35]	valid_0's auc: 0.788149
[40]	valid_0's auc: 0.790025
[45]	valid_0's auc: 0.79149
[50]	valid_0's auc: 0.792436
[55]	valid_0's auc: 0.792823
[60]	valid_0's auc: 0.793624
[65]	valid_0's auc: 0.794079
[70]	valid_0's auc: 0.79504
[75]	valid_0's auc: 0.795803
[80]	valid_0's auc: 0.796061
[85]	valid_0's auc: 0.796556
[90]	valid_0's auc: 0.796902
[95]	valid_0's auc: 0.797175
[100]	valid_0's auc: 0.797395
[105]	valid_0's auc: 0.797514
[110]	valid_0's auc: 0.797812
[115]	valid_0's auc: 0.798705
[120]	valid_0's auc: 0.798955
[125]	valid_0's auc: 0.799193
[130]	valid_0's auc: 0.799551
[135]	valid_0's auc: 0.799703
[140]	valid_0's auc: 0.799863
[145]	valid_0's auc: 0.800065
[150]	valid_0's auc: 0.800177
[155]	valid_0's auc: 0.800381
[160]	valid_0's auc: 0.800573
[1

In [18]:
pred_test = lgbm_model.predict(X_test)

The auc of prediction in test data is 0.8018.

In [19]:
roc_auc_score(y_test, pred_test)

0.8018124860426501

Now let me check for the feature importance, msno, artist_name and song_id are three most important variables.

In [20]:
pd.DataFrame([lgbm_model.feature_name(), lgbm_model.feature_importance()]).T

,0,1
0,msno,8010
1,song_id,2864
2,source_system_tab,41
3,source_screen_name,154
4,source_type,126
5,city,27
6,bd,8
7,gender,0
8,registered_via,13
9,membership_days,17


## Another method to deal with data

To try other models, I have to deal with the categorical variables first. Here I use one hot encoder for some important categorical variables.

In [21]:
del X_train, X_test, y_train, y_test

Let's check for number of level in each categorical variable.

In [22]:
for i in list(train.select_dtypes(include=['category']).columns):
    print(i + '  ' + str(len(train[i].unique())))

msno  30755
song_id  359966
source_system_tab  9
source_screen_name  21
source_type  13
city  21
gender  3
registered_via  5
genre_ids  573
artist_name  40583
composer  76065
lyricist  33889
language  11
country_code  111
registrant_code  6457


Based on the former result and business sense, I choose 'source_type', 'gender', 'language' three variables to do one-hot-encoder. For the rest of  categorical variables, simply use numbers to replace with label.

In [23]:
# One Hot Encoder
encoder = ce.OneHotEncoder(cols=['source_type','gender','language'],use_cat_names=True).fit(train)
train2 = encoder.transform(train)

In [24]:
## merge new features
train2 = train2.merge(msno_song_count, on='msno', how='left')
train2 = train2.merge(song_msno_count, on='song_id', how='left')

In [25]:
# Encoding categorical features
for col in train2.select_dtypes(include=['category']).columns:
    if col != 'msno ' and col != 'song_id ':
        train2[col] = train2[col].cat.codes

After a series of data cleaning procedures, now we have 52 variables.

In [26]:
train2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7377418 entries, 0 to 7377417
Data columns (total 52 columns):
msno                                  int16
song_id                               int32
source_system_tab                     int8
source_screen_name                    int8
source_type_album                     int64
source_type_artist                    int64
source_type_listen-with               int64
source_type_local-library             int64
source_type_local-playlist            int64
source_type_online-playlist           int64
source_type_radio                     int64
source_type_song                      int64
source_type_song-based-playlist       int64
source_type_top-hits-for-artist       int64
source_type_topic-article-playlist    int64
source_type_my-daily-playlist         int64
source_type_no data                   int64
target                                int64
city                                  int8
bd                                    int64
gender_fem

In [27]:
factor_Y = 'target'
factor_X = list(train2.columns)
factor_X.remove(factor_Y)

I redo the data split again as the former split with random_state=0.

In [28]:
## data split
X_train, X_test, y_train, y_test = train_test_split(train2[factor_X], train2[factor_Y], test_size=0.2, random_state=0)

## retrain Light GBM

I retrain the Light GBM model with the same parameters to see which kind of data cleaning method is more suitable.

In [29]:
lgb_X_train, lgb_X_val, lgb_y_train, lgb_y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
lgb_train = lgb.Dataset(lgb_X_train, lgb_y_train)
lgb_val = lgb.Dataset(lgb_X_val, lgb_y_val)
lgb_test = lgb.Dataset(X_test, y_test)

In [30]:
lgbm_model = lgb.train(params, train_set = lgb_train, valid_sets = lgb_val, verbose_eval=5, early_stopping_rounds=5)

Training until validation scores don't improve for 5 rounds.
[5]	valid_0's auc: 0.738891
[10]	valid_0's auc: 0.754711
[15]	valid_0's auc: 0.767772
[20]	valid_0's auc: 0.773389
[25]	valid_0's auc: 0.780043
[30]	valid_0's auc: 0.784037
[35]	valid_0's auc: 0.786373
[40]	valid_0's auc: 0.78863
[45]	valid_0's auc: 0.790563
[50]	valid_0's auc: 0.792077
[55]	valid_0's auc: 0.793203
[60]	valid_0's auc: 0.79386
[65]	valid_0's auc: 0.794911
[70]	valid_0's auc: 0.795807
[75]	valid_0's auc: 0.796221
[80]	valid_0's auc: 0.797006
[85]	valid_0's auc: 0.797122
[90]	valid_0's auc: 0.797985
[95]	valid_0's auc: 0.798938
[100]	valid_0's auc: 0.799303
Did not meet early stopping. Best iteration is:
[99]	valid_0's auc: 0.799314


The auc of prediction in test data is 0.7990.

In [31]:
pred_test = lgbm_model.predict(X_test)
roc_auc_score(y_test, pred_test)

0.799091623828119

Now let me check for the feature importance again, song_count, membership_days and msno are three most important variables.

In [32]:
pd.DataFrame([lgbm_model.feature_name(), lgbm_model.feature_importance()]).T

,0,1
0,msno,5160
1,song_id,2027
2,source_system_tab,38
3,source_screen_name,139
4,source_type_album,5
5,source_type_artist,0
6,source_type_listen-with,2
7,source_type_local-library,14
8,source_type_local-playlist,33
9,source_type_online-playlist,16


## model 2 TPOT

TPOT stands for Tree-based Pipeline Optimization Tool. TPOT is a Python Automated Machine Learning tool that optimizes machine learning pipelines using genetic programming. Its modeling procedure is showed as follow

![picture](./tpot.png)

TPOT completes feature engineering, feature selection, model selection and parameter optimization automatedly. Here we set parameters generations=5, population_size=20, cv=5, max_time_mins=10 and then trained the model.

In [33]:
tpot = TPOTClassifier(generations=5, population_size=20, cv=5, max_time_mins=5, verbosity=2, random_state=0)
tpot.fit(X_train, y_train)


9.699088399999999 minutes have elapsed. TPOT will close down.
TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.

Best pipeline: LogisticRegression(input_matrix, C=0.1, dual=False, penalty=l2)


TPOTClassifier(generations=1000000, max_time_mins=5, population_size=20,
               random_state=0, verbosity=2)

In [34]:
pred_test_tpot = tpot.predict(X_test)
roc_auc_score(y_test, pred_test_tpot)

0.5623081767297738

## model 3 Adaboost

I want to train another boosting model to compare with the Light GBM. However the training of Adaboost on the whole dataset costs too much time, so here I only choose a subset(10%) of train data to train the model.

In [38]:
X_train_s = X_train.sample(frac=0.1, random_state=0)
y_train_s = y_train.sample(frac=0.1, random_state=0)

In [39]:
adaboost = AdaBoostClassifier(n_estimators=50, random_state=0)
adaboost.fit(X_train_s, y_train_s)

AdaBoostClassifier(random_state=0)

In [40]:
pred_test_adaboost = adaboost.predict(X_test)
roc_auc_score(y_test, pred_test_adaboost)

0.6460334790006289

## Conclusion

In this project, I mainly try Light GBM in two different kind of data cleaning method, TPOT and other tree model like Adaboost and random forest. It turns out that Light GBM is the best one with auc 0.8018 and 0.7990 in the test data, which implies that these two method of data cleaning do not have too much difference for Light GBM in terms of auc performance. The training of TPOT takes a long time but only get a very bad peformance auc 0.5623. Both adaboost and random forest are not suitable for this problem since their training process cost too much time. Adaboost with 10% of training data only got 0.6460 auc on the test data, which is not good. Comparing to adaboost, Light GBM not only has a fast training speed, but also achieves a higher accuracy. <br>
Finally, I can get the conclusion that Light GBM is quite a good model to deal with hugh dataset in a fast and accurate way.